In [186]:
import gym
import numpy as np
import random
from itertools import chain, combinations
import itertools
from gym import spaces
import torch
import gym
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch import nn
from collections import deque,namedtuple

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def encode(lst):
    deck = []
    for pinta in ['O','B','C','E']:
        for n in range(10):
            deck.append(str(n)+pinta)
    encoded = [0]*40
    for card_lst in lst:
        for card in card_lst:
            encoded[deck.index(card)]=1
    return encoded

class president_game:
    def __init__(self,n_j):
        self.players_data = []
        self.stack = []
        self.history = []
        self.n_j = n_j
        self.deck = []
        self.active_player = None
        self.last_player = None
        for pinta in ['O','B','C','E']:
            for n in range(10):
                self.deck.append(str(n)+pinta)
    
    def deal(self):
        self.history = []
        self.players_data = []
        self.stack = []
        available_indices = list(range(40))
        for player in range(self.n_j):
            sample = random.sample(available_indices,int(40/self.n_j))
            self.players_data.append([self.deck[ind] for ind in sample])
            available_indices = list(set(available_indices) - set(sample))
            
    def active_player_0(self):
        if self.active_player == None:
            for player in range(self.n_j):
                if '0O' in self.players_data[player]:
                    self.active_player = player
        
    def possible_plays(self, player = None):
        if player == None:
            active_hand = self.players_data[self.active_player]
        else:
            active_hand = self.players_data[player]
        curr_quantity = len(self.stack)
        if curr_quantity == 0:
            if ('0O' in active_hand):
                pos_plays = [lst for lst in list(powerset([c for c in active_hand if c[0]=='0'])) if ('0O' in lst)]
                #for p in range(len(pos_plays)):
                    #print(str(p)+': ',pos_plays[p])
                return pos_plays          
        elif self.stack == ['*']:
            pos_plays = [lst for lst in list(powerset(active_hand)) if len(list(set([c[0] for c in lst])))==1]
            pos_plays.append([])
            return pos_plays
        else:
            curr_number = int(self.stack[0][0])
            pos_plays = [lst for lst in itertools.combinations(active_hand, curr_quantity) if (len(list(set([c[0] for c in lst])))==1) and (int(lst[0][0]) > curr_number)]
            #print([lst for lst in itertools.combinations(active_hand, curr_quantity) if (len(list(set([c[0] for c in lst])))==1)])
            pos_plays.append([])
            return pos_plays
     
    def play_select(self, selection = None,render = False):
        if self.last_player == self.active_player:
            self.stack = ['*']
        pos_plays = self.possible_plays()
        if render == True:
            for ind in range(len(pos_plays)):
                print(ind,': ',pos_plays[ind])
        if selection == None:
            selection = int(input('Que jugada elige :'))
        if pos_plays[selection] != []:
            self.stack = list(pos_plays[selection])
            self.history.append(list(pos_plays[selection]))
            self.last_player = self.active_player
            self.players_data[self.active_player] = list(set(self.players_data[self.active_player])-set(pos_plays[selection]))
        self.active_player = (self.active_player + 1)% self.n_j
        return self
        
    def hand_eval(self, player):
        hand_evaluation = 0
        for c in self.players_data[player]:
            hand_evaluation += -int(c[0])-1
        return hand_evaluation
    
    

C:\Users\Alonso\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [175]:
class president_gym(gym.Env):
    def __init__(self,n_players):
        super().__init__()
        self.n_players = n_players
        self.observation_space = spaces.MultiDiscrete([2]*80+[4,10])
        self.action_space = spaces.Discrete(41)
        self.placements = []
        
    def reset(self):
        self.game = president_game(self.n_players)
        self.game.deal()
        self.game.active_player_0()
        self.history = []
        act = self.game.active_player
        act_hand = self.game.players_data[act]
        self.state = encode([act_hand,self.game.history])
        return self 
        
    def step(self,action,reward='placement'):
        still_playing = [player for player in range(self.n_players) if len(self.game.players_data[player])!=0] 
        if len(still_playing)<=1:
            if len(still_playing)==1:
                self.placement.append(still_playing[0])
            done = True
        self.game.play_select(action)
        self.state = encode([act_hand,history])
        info = ''
        
        if reward == 'placement':
            if done == True:
                reward = -100 * (placement.index(self.game.active_player))
            else:
                reward = 0
        return self.state, reward, done, info
        
    def render(self, godmode=False):
        print(f'Active player is player {self.game.active_player} with hand {self.game.players_data[self.game.active_player]}')
        print(f'With possible plays :{self.game.possible_plays()}')
        print(f'Game history so far is: {self.game.history}')
        

In [159]:
president_train = president_gym(4)
president_seed = president_gym(4)
president_train.reset()
act = president_train.game.active_player
act_hand = president_train.game.players_data[act]
print(f'Jugador {act} tiene mano {act_hand}')
state = [act_hand,president_train.history]
encoded_state = encode(state)
print(president_train.game.possible_plays())
print(encoded_state)
pos_plays = president_train.game.possible_plays()

Jugador 3 tiene mano ['5B', '0E', '7O', '6O', '0O', '1B', '2E', '3E', '5O', '3B']
[('0O',), ('0E', '0O')]
[1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]


In [182]:
plays = [[0,0]]
for number in range(0,10):
    for amount in range(1,5):
        plays.append([amount,number])

def encode_play(lst):
    pos_plays = []
    for play in lst:
        if play == []:
            pos_plays.append([0,0])
        elif play != []:
            if len(set([p[0] for p in play])) == 1:
                pos_plays.append([len(play),int(play[0][0])])
    return pos_plays

def masked_weights(pos_plays,weights,plays=plays):
    return [weights[ind]*(plays[ind] in pos_plays)-100000*(plays[ind] not in pos_plays) for ind in range(len(plays))]



C:\Users\Alonso\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [183]:
class DQN(nn.Module):
    def __init__(self,state_space_dim,action_space_dim):
        super().__init__()
        
        self.linear = nn.Sequential(
            nn.Linear(state_space_dim,64),
            nn.ReLU(),
            nn.Linear(64,64*2),
            nn.ReLU(),
            nn.Linear(64*2,action_space_dim)
        )
        
    def forward(self,x):
        x = x.to(device)
        return self.linear(x)

class ReplayMemory(object):
    def __init__(self,capacity):
        self.memory = deque(maxlen = capacity)
    def push(self,state,action,next_state,reward):
        self.memory.append((state,action,next_state,reward))
    def sample(self,batch_size):
        batch_size = min(batch_size,len(self))
        return random.sample(self.memory,batch_size)
    def __len__(self):
        return len(self.memory)
    

def choose_action_epsilon_greedy(net,state,epsilon):
    
    if epsilon > 1 or epsilon < 0:
        raise Exception('The epsilon value must be between 0 and 1.')
    
    with torch.no_grad():
        net.eval()
        state = torch.tensor(state,dtype=torch.float32)
        net_out = net(state)
    
    best_action = int(net_out.argmax())
    
    action_space_dim = net_out.shape[-1]
    
    if random.random() < epsilon:
        non_optimal_actions = [a for a in range(action_space_dim) if a!= best_action]
        action = random.choice(non_optimal_actions)
    else:
        action = best_action
        
    return action, net_out.cpu().numpy()

def choose_action_softmax(net, state, temperature):
    
    if temperature < 0:
        raise Exception('The temperature value must be greater than or equal to 0 ')
        
    # If the temperature is 0, just select the best action using the eps-greedy policy with epsilon = 0
    if temperature == 0:
        return choose_action_epsilon_greedy(net, state, 0)
    
    # Evaluate the network output from the current state
    with torch.no_grad():
        net.eval()
        state = torch.tensor(state, dtype=torch.float32)
        net_out = net(state)

    # Apply softmax with temp
    temperature = max(temperature, 1e-8) # set a minimum to the temperature for numerical stability
    softmax_out = nn.functional.softmax(net_out/temperature, dim=0).cpu().numpy()
                
    # Sample the action using softmax output as mass pdf
    all_possible_actions = np.arange(0, softmax_out.shape[-1])
    # this samples a random element from "all_possible_actions" with the probability distribution p (softmax_out in this case)
    action = np.random.choice(all_possible_actions,p=softmax_out)
    
    return action, net_out.cpu().numpy()
    
    

In [184]:
# Initialize the environment

N_PLAYERS = 4
NUM_GAMES = 10

game = president_gym(4)
game = game.reset()
game.render()

active_player = game.game.active_player
hand = game.game.players_data[active_player]
pos_plays = game.game.possible_plays()
stack = game.game.stack
history = game.game.history

print(pos_plays)

net_input_stack = encode_play([stack])
net_input_hand = encode([hand])
net_input_history = encode([history])
net_input_pos_plays = encode_play(pos_plays)
#print(net_input_hand + net_input_history + net_input_stack)

net_input = torch.tensor(net_input_hand + net_input_history + net_input_stack[0], dtype = torch.float32)
input_dim = net_input.shape[-1]
output_dim = 

SyntaxError: invalid syntax (<ipython-input-184-b7ffcde1276e>, line 26)

In [187]:
# Initialize the environment
import copy

device = 'cpu'

plays = [[0,0]]
for number in range(0,10):
    for amount in range(1,5):
        plays.append([amount,number])

N_PLAYERS = 4
NUM_GAMES = 1
MAX_NUMBER_GAME_STEPS = 10
PLAY_TYPE = 'self-play'
TARGET_NET_UPDATE_STEPS = 10
REPLAY_MEMORY_CAPACITY = 10000
LR = 1e-3
game = president_gym(N_PLAYERS)
replay_mem = ReplayMemory(REPLAY_MEMORY_CAPACITY)

policy_net = DQN(82+N_PLAYERS-1, 41).to(device)

target_net = DQN(82+N_PLAYERS-1, 41).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = torch.optim.Adam(policy_net.parameters(),lr = LR)
loss_fn = nn.SmoothL1Loss()

for game_index in range(NUM_GAMES):
    
    game = game.reset()
    game_done = False
    game_steps = 0
    
    while not game_done and game_steps <= MAX_NUMBER_GAME_STEPS:
        
        still_playing = [hand for hand in game.game.players_data if len(hand)!=0]
        
        if len(still_playing)<=1:
            
            placements += [game.game.players_data.index(still_playing[0])]
            game_done = True
            
        else:
            
            active_player = game.game.active_player
            hand = game.game.players_data[active_player]
            other_hand_lengths = [len(game.game.players_data[ind]) for ind in range(N_PLAYERS) if ind != active_player]
            pos_plays = game.game.possible_plays()
            encoded_pos_plays = encode_play(pos_plays)
            stack = game.game.stack
            history = game.game.history

            net_input_stack = encode_play([stack])
            net_input_hand = encode([hand])
            net_input_history = encode(history)
            net_input_pos_plays = encode_play(pos_plays)
            
            with torch.no_grad():
                net_input = torch.tensor(net_input_hand + net_input_history + other_hand_lengths + net_input_stack[0],dtype = torch.float32)
                net_output = policy_net(net_input)
                masked_output = masked_weights(encoded_pos_plays,net_output.detach().cpu().numpy())
                print('Possible plays :',encoded_pos_plays)
                print('Masked neural net output :',masked_output)
                print(plays[np.argmax(masked_output)])
                #print(best_pos_play)
                #print(net_output)
                print(encoded_pos_plays.index(plays[np.argmax(masked_output)]))
                game.game.play_select(encoded_pos_plays.index(plays[np.argmax(masked_output)]))
                print(f'history: {game.game.history}')
        game_steps +=1
            
            
            

Possible plays : [[1, 0], [2, 0], [2, 0], [3, 0]]
Masked neural net output : [-100000.0, -0.014269459992647171, -0.02080719918012619, 0.1937359869480133, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0]
[3, 0]
3
history: [['0E', '0C', '0O']]
Possible plays : [[0, 0]]
Masked neural net output : [0.11270283907651901, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -100000.0, -10000

In [57]:
NUM_GAMES = 100

MAX_GAME_STEPS = 200

Q_net = DQN(2,41)

for game_index in range(NUM_GAMES):
    pass

state = [1,2]

net_input = torch.tensor(state,dtype=torch.float32)
net_output = Q_net.linear.forward(net_input).detach().numpy()

print(net_output,np.argmax(net_output))

[ 0.06916992 -0.00265957 -0.4783727   0.03750735 -0.24600804  0.10422891
  0.0759403  -0.15358374  0.08696938 -0.11341097  0.2536936   0.01497917
 -0.5329273   0.29117742 -0.2256351  -0.29870304 -0.10671773  0.26369077
  0.246369   -0.28758222 -0.25909916  0.26477763  0.11355833  0.02271875
 -0.4679014  -0.05509595  0.04152231  0.30295223 -0.0774397   0.01080476
  0.06304856  0.05994857  0.00648457 -0.03037679 -0.08925813  0.03488246
 -0.04815163 -0.11345432  0.01472715  0.0635439  -0.26832205] 27


C:\Users\Alonso\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


C:\Users\Alonso\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3

In [ ]:
### Define exploration profile
initial_value = 5
num_iterations = 800
exp_decay = np.exp(-np.log(initial_value) / num_iterations * 6) 
exploration_profile = [initial_value * (exp_decay ** i) for i in range(num_iterations)]


# environment
env = gym.make('Acrobot-v1') 
env.seed(0) # Set a random seed for the environment 

state_space_dim = env.observation_space.shape[0]
action_space_dim = env.action_space.n
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"STATE SPACE SIZE: {state_space_dim}")
print(f"ACTION SPACE SIZE: {action_space_dim}")

# Set random seeds
torch.manual_seed(0)
np.random.seed(0)

gamma = 0.99  
replay_memory_capacity = 10000   
lr = 1e-3
target_net_update_steps = 10   
batch_size = 256   
bad_state_penalty = 0   
min_samples_for_training = 1000   

# replay memory
replay_mem = ReplayMemory(replay_memory_capacity)    

# policy network
policy_net = DQN(state_space_dim, action_space_dim).to(device)

# target network with the same weights of the policy network
target_net = DQN(state_space_dim, action_space_dim).to(device)
target_net.load_state_dict(policy_net.state_dict()) # This will copy the weights of the policy network to the target network

optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr) # The optimizer will update ONLY the parameters of the policy network

